In [1]:
# default_exp data.clean_rows

# Clean dataset
> remove duplicates, N/A and calculate bad order cancellations

In [1]:
# hide
NB_NAME = '02_df_clean_rows'
OUT_DATA_PATH = f'../data/{NB_NAME}'

In [2]:
#hide
%load_ext autoreload
%autoreload 2

In [3]:
# export
import logging
import pandas as pd

In [4]:
# hide
logging.basicConfig(level=logging.INFO)

In [9]:
# export

def clean_drop_na(df: pd.DataFrame) -> pd.DataFrame:
    df = df.dropna(axis=0, subset=['CustomerID'])
    return df

def clean_drop_duplicates(df: pd.DataFrame) -> pd.DataFrame:
    df = df.drop_duplicates()
    return df

In [10]:
# export

def clean_remove_bad_order_cancellations(df: pd.DataFrame) -> pd.DataFrame:
    """
    Returns a copy of `df` that doesn't include the following orders:
      1. a cancel order exists without counterpart,
      2. a order for which there exists at least one counterpart with exactly the same quantity.
    NOTE: time complexity is O(n).
    """
    
    df_cleaned = df.copy(deep=True)
    df_cleaned['QuantityCanceled'] = 0

    entry_to_remove = [] ; doubtfull_entry = []

    for index, col in  df.iterrows():
        if col['Quantity'] > 0 or col['Description'] == 'Discount':
            continue        
        df_test = df[(df['CustomerID'] == col['CustomerID']) &
                     (df['StockCode']  == col['StockCode']) & 
                     (df['InvoiceDate'] < col['InvoiceDate']) & 
                     (df['Quantity']   > 0)].copy()
        
        # Cancelation WITHOUT counterpart
        if df_test.shape[0] == 0: 
            doubtfull_entry.append(index)
        
        # Cancelation WITH a counterpart
        elif df_test.shape[0] == 1: 
            index_order = df_test.index[0]
            df_cleaned.loc[index_order, 'QuantityCanceled'] = -col['Quantity']
            entry_to_remove.append(index)        
        
        # Various counterparts exist in orders: we delete the last one
        elif df_test.shape[0] > 1:
            df_test.sort_index(axis=0 ,ascending=False, inplace=True)
            for ind, val in df_test.iterrows():
                if val['Quantity'] < -col['Quantity']: continue
                df_cleaned.loc[ind, 'QuantityCanceled'] = -col['Quantity']
                entry_to_remove.append(index) 
                break            
    
    logging.info("entry_to_remove: {}".format(len(entry_to_remove)))
    logging.info("doubtfull_entry: {}".format(len(doubtfull_entry)))

    df_cleaned.drop(entry_to_remove, axis=0, inplace=True)
    df_cleaned.drop(doubtfull_entry, axis=0, inplace=True)
    
    remaining_entries = df_cleaned[(df_cleaned['Quantity'] < 0) & (df_cleaned['StockCode'] != 'D')]
    logging.info("nb of entries to delete: {}".format(remaining_entries.shape[0]))
    logging.info(str(remaining_entries[:5]))
    
    df_cleaned['TotalPrice'] = df_cleaned['UnitPrice'] * (df_cleaned['Quantity'] - df_cleaned['QuantityCanceled'])
    logging.info(str(df_cleaned.sort_values('CustomerID')[:5]))
    
    return df

In [11]:
# export

def clean_rows(df: pd.DataFrame) -> pd.DataFrame:
    df = clean_drop_na(df)
    df = clean_drop_duplicates(df)
    df = clean_remove_bad_order_cancellations(df)
    return df

In [14]:
from elab2team2.df.load_split import load_csv

df = load_csv('../data/01_df_split_offline_online/no_live_data.csv')
clean_rows(df[:1000])

INFO:root:entry_to_remove: 1
INFO:root:doubtfull_entry: 8
INFO:root:nb of entries to delete: 0
INFO:root:Empty DataFrame
Columns: [InvoiceNo, StockCode, Description, Quantity, InvoiceDate, UnitPrice, CustomerID, Country, QuantityCanceled]
Index: []
INFO:root:    InvoiceNo StockCode                   Description  Quantity  \
208    536389     22191        IVORY DINER WALL CLOCK         2   
197    536389     22941  CHRISTMAS LIGHTS 10 REINDEER         6   
210    536389     22196  SMALL HEART MEASURING SPOONS        24   
209    536389     22195  LARGE HEART MEASURING SPOONS        24   
207    536389     22192         BLUE DINER WALL CLOCK         2   

            InvoiceDate  UnitPrice CustomerID    Country  QuantityCanceled  \
208 2010-12-01 10:03:00       8.50      12431  Australia                 0   
197 2010-12-01 10:03:00       8.50      12431  Australia                 0   
210 2010-12-01 10:03:00       0.85      12431  Australia                 0   
209 2010-12-01 10:03:00   

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
...,...,...,...,...,...,...,...,...
995,536520,22469,HEART OF WICKER SMALL,1,2010-12-01 12:43:00,1.65,14729,United Kingdom
996,536520,22100,SKULLS SQUARE TISSUE BOX,1,2010-12-01 12:43:00,1.25,14729,United Kingdom
997,536520,22096,PINK PAISLEY SQUARE TISSUE BOX,1,2010-12-01 12:43:00,1.25,14729,United Kingdom
998,536520,22583,PACK OF 6 HANDBAG GIFT BOXES,1,2010-12-01 12:43:00,2.55,14729,United Kingdom


In [ ]:
# hide
if False:  # only to be run manually
    df_cleaned = clean_rows(df)
    !mkdir -p {OUT_DATA_PATH}
    df_cleaned.to_csv(f'{OUT_DATA_PATH}/no_live_data_cleaned.csv')